In [1]:
from collections import defaultdict
import numpy as np
import scipy.io as sp
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Helvetica" 

from cmcrameri import cm
from scipy.signal import savgol_filter
import pySCION_plotting
import scipy.stats as stats
from scipy.interpolate import interp1d

In [2]:
def get_wasserstein_distance(x1_mean, x1_min, x1_max, x2):
    # print(x2)
    mu = x1_mean
    no_of_bins = len(x2)*0.1

    # the diff should be the same between mean and ± std, as it's a normal/non skewed distribution
    sigma = np.mean(np.diff([x1_min, x1_mean, x1_max]))
    if sigma == 0:
        sigma = 0.01
    # print(x1_min, x1_mean, x1_max)
    # evenly spaced values between our stds for SCOTESE temp
    range_x1, step_x1 = np.linspace(
        mu-sigma, mu+sigma, int(no_of_bins), retstep=True)
    # probability of each evenly spaced value occurring
    temp_hist_x1 = stats.norm.pdf(range_x1, mu, sigma)
    # have to multiply by the stepsize to get proper probability
    # should give 0.687, since this is just the values between ± 1 std
    u_weights = temp_hist_x1*step_x1

    # now x2, our model results, if using full distribution (i.e. each individual model run)
    counts_x2, bounds_x2 = np.histogram(x2, bins=int(no_of_bins))
    v_weights = counts_x2/np.sum(counts_x2)
    # get centres of the histograms bins so we know their value
    centres_x2 = 0.5*(bounds_x2[1:]+bounds_x2[:-1])
    # print(probability_x1)
    wasserstein_distance = stats.wasserstein_distance(range_x1,
                                                      centres_x2,
                                                      u_weights=u_weights,
                                                      v_weights=v_weights)

    return wasserstein_distance

In [ ]:
#make color map for plots
cmap = cm.batlow
colors = cmap(np.linspace(0, 1, 7))
ordered_colors = [[0,0,0,1], #default
                  colors[1],#arc
                  colors[3],#biological
                  colors[4],#degass
                  colors[5],#pgeog
                  colors[0]]#suture                 
cmaps = mpl.colors.ListedColormap(ordered_colors)
cmaps

In [4]:
# Load proxy data
# note, all proxy data runs from present-day back in time
# ice data, compiled in 'plot_proxy_data.ipynb'
import pickle
ice_data = pd.read_csv('./data/M2021_iceline_abs.csv')

# D18O tropical temp
with open('./data/d18O_dat.pkl', 'rb') as handle:
    O2_dict = pickle.load(handle)

# Load GAST (scotese curve, from VDM paper though)
file_to_open_VDMEER = './data/vandermeer_2022_estimates.xlsx'
vdMEER_data = pd.read_excel(file_to_open_VDMEER, header=2)
vdMEER_GAT = vdMEER_data[['Ma',
                          'GAT_degC',
                          'GAT_degC.1',
                          'GAT_degC.2']]

# Load pCO2 data
#### load geochem data
file_to_open_GEOCHEM = './data/geochem_data_2024.mat'
geochem_data = sp.loadmat(file_to_open_GEOCHEM)

# load uncertainty on data
with open(f'./data/proxy_data_best_fit.pkl', 'rb') as handle:
    proxy_data_best_fit = pickle.load(handle)

# load Judd data
judd2024_gast = pd.read_csv('./data/PhanDA_GMSTandCO2_percentiles.csv')
judd2024_gast_x = judd2024_gast['AverageAge']*-1
judd2024_gast_y50 = judd2024_gast['GMST_50']
judd2024_gast_y84 = judd2024_gast['GMST_95']
judd2024_gast_y16 = judd2024_gast['GMST_05']

In [ ]:
#load your sensitivity files here, output from publication available on zenodo (too big for github)

#files = ['LIP_test_iter=1000_2024Sep03']
#files = ['ALL_OFF_iter=10000_2024Apr29',
#         'ARC_ONLY_iter=1000_2024Apr29',
#         'DEGASS_ONLY_iter=1000_2024Apr29',
#         'PGEOG_ONLY_iter=1000_2024Apr29',
#         'SUTURE_ONLY_iter=1000_2024Apr29']
files = ['ALL_ON_iter=1000_2024Apr29',
         'ARC_OFF_iter=1000_2024Apr29',
         'DEGASS_OFF_iter=1000_2024Apr29',
         'PGEOG_OFF_iter=1000_2024Apr29',
         'SUTURE_OFF_iter=1000_2024Apr29']

results_dict = defaultdict(dict)
# save?
save = False
plot_mean = True
times = np.arange(-420, 1, 1)

# proxy data are saved from present-day backwards, but results are
# forward in time, so reverse them here
ice_proxy = [proxy_data_best_fit['ice_y_smooth'][:421][::-1],
             times]
gast_proxy = [vdMEER_GAT['GAT_degC'][1:422].values.astype(float)[::-1],
              vdMEER_GAT['GAT_degC.1'][1:422].values.astype(float)[::-1],
              vdMEER_GAT['GAT_degC.2'][1:422].values.astype(float)[::-1],
              times]
gast2_proxy = [judd2024_gast_y50,
               judd2024_gast_y16,
               judd2024_gast_y84,
               judd2024_gast_x]
sat_proxy = [proxy_data_best_fit['o2_y_mean'][:421][::-1],
             proxy_data_best_fit['o2_y_low'][:421][::-1],
             proxy_data_best_fit['o2_y_upper'][:421][::-1],
             times]
co2_proxy = [proxy_data_best_fit['co2_y_mean'][:421][::-1],
             proxy_data_best_fit['co2_y_low'][:421][::-1],
             proxy_data_best_fit['co2_y_upper'][:421][::-1],
             times]

ice_distances = []
sat_distances = []
gast_distances = []
co2_distances = []

for ind, filename in enumerate(files[:]):
    results = pySCION_plotting.get_results(filename)
    if np.isclose(results.time_myr, results.time_myr[0]).all():
        results_interp_ice = interp1d(results.time_myr[0], results.iceline, axis=1)
        results_interp_sat = interp1d(results.time_myr[0], results.sat_tropical, axis=1)
        results_interp_tgast = interp1d(results.time_myr[0], results.T_gast, axis=1)
        results_interp_co2 = interp1d(results.time_myr[0], results.co2ppm, axis=1)
    else:
        print('times arent close enough')

    color = ordered_colors[0] # just use black for everything
    ice_distance = np.zeros_like(times).astype(float) 
    sat_distance = np.zeros_like(times).astype(float) 
    gast_distance = np.zeros_like(times).astype(float) 
    co2_distance = np.zeros_like(times).astype(float) 
    for time_ind, time in enumerate(times[:]):

        ice_distance[time_ind] = get_wasserstein_distance(ice_proxy[0][time_ind],
                                                          ice_proxy[0][time_ind],
                                                          ice_proxy[0][time_ind],
                                                          results_interp_ice(time)[:, 1])
        
        sat_distance[time_ind] = get_wasserstein_distance(sat_proxy[0][time_ind],
                                                          sat_proxy[1][time_ind],
                                                          sat_proxy[2][time_ind],
                                                          results_interp_sat(time))
        
        gast_distance[time_ind] = get_wasserstein_distance(gast_proxy[0][time_ind],
                                                           gast_proxy[1][time_ind],
                                                           gast_proxy[2][time_ind],
                                                           results_interp_tgast(time))
        
        co2_distance[time_ind] = get_wasserstein_distance(co2_proxy[0][time_ind],
                                                          co2_proxy[1][time_ind],
                                                          co2_proxy[2][time_ind],
                                                          results_interp_co2(time))
    ice_distances.append(ice_distance)
    sat_distances.append(sat_distance)
    gast_distances.append(gast_distance)
    co2_distances.append(co2_distance)


    results_dict['iceline'][filename] = np.mean(
        results.iceline, axis=0), np.mean(ice_distance), ice_distance
    results_dict['sat'][filename] = np.mean(
        results.sat_tropical, axis=0), np.mean(sat_distance), sat_distance
    results_dict['gast'][filename] = np.mean(
        results.T_gast, axis=0), np.mean(gast_distance), gast_distance
    results_dict['co2'][filename] = np.mean(
        results.co2ppm, axis=0), np.mean(co2_distance), co2_distance
    
    print(ind)
    
    if ind == 0:
        # default results are our 'all on' results
        default_results = results
        pySCION_plotting.make_global_temp_plot(results, [gast_proxy, gast2_proxy], gast_distance,
                                               color, filename, default_results=False,
                                               plot_mean=plot_mean, save=save)
    else:
        pySCION_plotting.make_global_temp_plot(results, [gast_proxy, gast2_proxy], gast_distance, 
                                               color, filename, default_results=default_results,  
                                               plot_mean=plot_mean, save=save)

    pySCION_plotting.make_equtorial_temp_plot(results, O2_dict, sat_proxy, sat_distance,
                                              color, filename, plot_mean=plot_mean, save=save)
    
    pySCION_plotting.make_ice_plot(results, ice_data, ice_proxy, ice_distance, color,
                                   filename, plot_mean=plot_mean, save=save)
    
    pySCION_plotting.make_CO2_plot(results, geochem_data, co2_proxy, co2_distance,
                                   color, filename, plot_mean=plot_mean, save=save)
    
    print(filename, np.mean(gast_distance))
    

In [7]:
ordered_colors = ['#010101',
                  '#1B4F62',
                  '#838233',
                  '#D2933F',
                  '#F8AB9E',
                  '#010101']

ordered_lws = [9,
               5,
               2,
               5,
               3,
               1]
ordered_lss = [(0, (1, 1)),
               'solid',
               'solid',
               (0, (5, 1)),
               (0, (5, 10)),
               'solid']

In [ ]:
files = ['ALL_OFF_iter=10000_2024Apr29',
        'ARC_ONLY_iter=1000_2024Apr29',
        'DEGASS_ONLY_iter=1000_2024Apr29',
        'PGEOG_ONLY_iter=1000_2024Apr29',
        'SUTURE_ONLY_iter=1000_2024Apr29',
         'ALL_ON_iter=1000_2024Apr29']
#plot figure 3
#default = ['ALL_ON_iter=1000_2024Apr29']
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, figsize=(16,16))
linewidth = 4
alpha = 0.2
proxy_alpha = 0.2
proxy_colour = '#450903'#'lightgray'
for ind, filename in enumerate(files[:]):
    
    results = pySCION_plotting.get_results(filename)
    #default_results = pySCION_plotting.get_results(default)

    if (filename == 'DEGASS_ONLY_iter=1000_2024Apr29') | (filename == 'ALL_ON_iter=1000_2024Apr29'):
        # ice data
        ice_upper_std = np.mean(results.iceline, axis=0)[:,1]+np.std(results.iceline, axis=0)[:,1]
        ice_lower_std = np.mean(results.iceline, axis=0)[:,1]-np.std(results.iceline, axis=0)[:,1]
        ice_mean = np.mean(results.iceline, axis=0)[:,1]

        ice_upper_std[ice_upper_std > 90] = 90
        ice_lower_std[ice_lower_std > 90] = 90
        ice_mean[ice_mean > 90] = 90
        ax1.plot(np.mean(results.time_myr, axis=0), ice_mean, c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax1.plot(np.mean(results.time_myr, axis=0), ice_upper_std, c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax1.plot(np.mean(results.time_myr, axis=0), ice_lower_std, c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax1.fill_between(np.mean(results.time_myr, axis=0),
                            ice_upper_std,
                            ice_lower_std,
                         color=ordered_colors[ind], alpha=alpha
                            )
        ax2.plot(np.mean(results.time_myr, axis=0), np.mean(results.co2ppm, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax2.plot(np.mean(results.time_myr, axis=0), np.mean(results.co2ppm, axis=0)+np.std(results.co2ppm, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax2.plot(np.mean(results.time_myr, axis=0), np.mean(results.co2ppm, axis=0)-np.std(results.co2ppm, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax2.fill_between(np.mean(results.time_myr, axis=0),
                         np.mean(results.co2ppm, axis=0)+np.std(results.co2ppm, axis=0),
                         np.mean(results.co2ppm, axis=0)-np.std(results.co2ppm, axis=0),
                         color=ordered_colors[ind], alpha=alpha)

        ax3.plot(np.mean(results.time_myr, axis=0), np.mean(results.sat_tropical, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax3.plot(np.mean(results.time_myr, axis=0), np.mean(results.sat_tropical, axis=0)+np.std(results.sat_tropical, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax3.plot(np.mean(results.time_myr, axis=0), np.mean(results.sat_tropical, axis=0)-np.std(results.sat_tropical, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax3.fill_between(np.mean(results.time_myr, axis=0),
                         np.mean(results.sat_tropical, axis=0)+np.std(results.sat_tropical, axis=0),
                         np.mean(results.sat_tropical, axis=0)-np.std(results.sat_tropical, axis=0),
                         color=ordered_colors[ind], alpha=alpha
                        )

        ax4.plot(np.mean(results.time_myr, axis=0), np.mean(results.T_gast, axis=0), c=ordered_colors[ind], label=filename, ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax4.plot(np.mean(results.time_myr, axis=0), np.mean(results.T_gast, axis=0)+np.std(results.T_gast, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax4.plot(np.mean(results.time_myr, axis=0), np.mean(results.T_gast, axis=0)-np.std(results.T_gast, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax4.fill_between(np.mean(results.time_myr, axis=0),
                         np.mean(results.T_gast, axis=0)+np.std(results.T_gast, axis=0),
                         np.mean(results.T_gast, axis=0)-np.std(results.T_gast, axis=0),
                         color=ordered_colors[ind], alpha=alpha
                        )

    else:
        color = ordered_colors[ind]
          # ice data
        ice_upper_std = np.mean(results.iceline, axis=0)[:,1]+np.std(results.iceline, axis=0)[:,1]
        ice_lower_std = np.mean(results.iceline, axis=0)[:,1]-np.std(results.iceline, axis=0)[:,1]
        ice_mean = np.mean(results.iceline, axis=0)[:,1]

        ice_upper_std[ice_upper_std > 90] = 90
        ice_lower_std[ice_lower_std > 90] = 90
        ice_mean[ice_mean > 90] = 90
        ax1.plot(np.mean(results.time_myr, axis=0), ice_mean, c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax2.plot(np.mean(results.time_myr, axis=0), np.mean(results.co2ppm, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax3.plot(np.mean(results.time_myr, axis=0), np.mean(results.sat_tropical, axis=0), c=ordered_colors[ind], ls=ordered_lss[ind], lw=ordered_lws[ind])
        ax4.plot(np.mean(results.time_myr, axis=0), np.mean(results.T_gast, axis=0), c=ordered_colors[ind], label=filename, ls=ordered_lss[ind], lw=ordered_lws[ind])



#ice proxy
ax1.matshow(ice_data, aspect=1.5, cmap=cm.bilbao, extent=[0, -540, 0, 90], alpha=alpha,
            vmax=10)

#co2 proxy
CO2 = geochem_data
cmap = cm.tokyo
colors1 = cmap(np.linspace(0, 1, 8))

pc1 = proxy_colour#colors1[0]
pc2 = proxy_colour#colors1[1]
pc3 = proxy_colour#colors1[3]
pc4 = proxy_colour#colors1[4]
pc5 = proxy_colour#colors1[5]
pc6 = proxy_colour#colors1[6]
pc7 = proxy_colour#colors[7]

ax2.errorbar(geochem_data['paleosol_age'].ravel(),
             geochem_data['paleosol_co2'].ravel(),
             yerr=np.abs(geochem_data['paleosol_high'].ravel() - geochem_data['paleosol_co2'].ravel(),
                         geochem_data['paleosol_co2'].ravel() - geochem_data['paleosol_low'].ravel()),
             fmt='o',  color=pc1, alpha=proxy_alpha, label='Palaeosol', zorder=1)

ax2.errorbar(geochem_data['alkenone_age'].ravel(),
             geochem_data['alkenone_co2'].ravel(),
             yerr=(geochem_data['alkenone_high'].ravel() - geochem_data['alkenone_co2'].ravel(),
                   geochem_data['alkenone_co2'].ravel() - geochem_data['alkenone_low'].ravel()),
             fmt='v',  color=pc2, alpha=proxy_alpha, label='Alkenone', zorder=1)

ax2.errorbar(geochem_data['boron_age'].ravel(),
             geochem_data['boron_co2'].ravel(),
             yerr=(geochem_data['boron_high'].ravel() - geochem_data['boron_co2'].ravel(),
                   geochem_data['boron_co2'].ravel() - geochem_data['boron_low'].ravel()),
             fmt='s',  color=pc3, alpha=proxy_alpha, label='Boron', zorder=1)

ax2.errorbar(geochem_data['stomata_age'].ravel(),
             geochem_data['stomata_co2'].ravel(),
             yerr=(geochem_data['stomata_high'].ravel() - geochem_data['stomata_co2'].ravel(),
                   geochem_data['stomata_co2'].ravel() - geochem_data['stomata_low'].ravel()),
             fmt='P',  color=pc4, alpha=proxy_alpha, label='Stomata', zorder=1)

ax2.errorbar(geochem_data['liverwort_age'].ravel(),
             geochem_data['liverwort_co2'].ravel(),
             yerr=(geochem_data['liverwort_high'].ravel() - geochem_data['liverwort_co2'].ravel(),
                   geochem_data['liverwort_co2'].ravel() - geochem_data['liverwort_low'].ravel()),
             fmt='x',  color=pc5, alpha=proxy_alpha, label='Liverwort', zorder=1)

ax2.errorbar(geochem_data['phytane_age'].ravel(),
             geochem_data['phytane_co2'].ravel(),
             yerr=(geochem_data['phytane_high'].ravel() - geochem_data['phytane_co2'].ravel(),
                   geochem_data['phytane_co2'].ravel() - geochem_data['phytane_low'].ravel()),
             fmt='D',  color=pc6, alpha=proxy_alpha, label='Phytane', zorder=1)

ax2.errorbar(geochem_data['phytoplankton_age'].ravel(),
             geochem_data['phytoplankton_co2'].ravel(),
             yerr=(geochem_data['phytoplankton_high'].ravel() - geochem_data['phytoplankton_co2'].ravel(),
                   geochem_data['phytoplankton_co2'].ravel() - geochem_data['phytoplankton_low'].ravel()),
             fmt='^',  color=pc7, alpha=proxy_alpha, label='Phytoplankton', zorder=1)

# EQAST
times = []
vals = []
for i in O2_dict:
        vals.append(O2_dict[i][0].values)
        times.append(O2_dict[i][1].values)

vals = np.concatenate(vals).ravel()
times = np.concatenate(times).ravel()
times = times[~np.isnan(vals)]*-1
vals = vals[~np.isnan(vals)]

hist = ax3.hist2d(times, vals, bins=(200, 100), norm=mpl.colors.LogNorm(),
        cmap=cm.bilbao, alpha=alpha,
                zorder=1)
# GAST
ax4.plot(gast_proxy[3].astype(float),
         gast_proxy[0].astype(float), lw=3, c=proxy_colour, alpha=proxy_alpha,
                zorder=1)

ax4.fill_between(gast_proxy[3].astype(float),
                 gast_proxy[1].astype(float),
                 gast_proxy[2].astype(float), color=proxy_colour, alpha=proxy_alpha,
                zorder=1)

ax4.plot(judd2024_gast_x,judd2024_gast_y50, color='indigo', lw=3, alpha=proxy_alpha, zorder=1)

ax4.fill_between(judd2024_gast_x,
                judd2024_gast_y84,judd2024_gast_y16, color='indigo', alpha=proxy_alpha, zorder=1)

axs = [ax1, ax2, ax3, ax4]
for ax in axs:
        ax.set_xlim([-420,0])   
        ax.grid()
        ax.tick_params(axis='both', labelsize=16)
ax1.set_ylabel('Latitude (°)', fontsize=20)
ax1.set_ylim([30,90])

ax2.set_yscale('log')
ax2.set_ylim([100, 10000])
ax2.set_ylabel(r'$Atmospheric\ CO_{2}\ (ppm)$', fontsize=20)

ax3.set_ylim([0,60])
ax3.set_ylabel('EQAST (°C)', fontsize=20)

ax4.set_ylabel('GAST (°C)', fontsize=20)
ax4.set_xlabel('Time (Ma)', fontsize=20)
ax4.set_ylim([0,35])

#fig.legend()
fig.tight_layout()

fig.savefig('./results/figures/Fig_4_driver_comparison_aug15.svg')
#450903

results dict structure:

    proxy_comparison (iceline, gast, tropical, co2)

        ISOLATED VARIABLE (bio, SF, AF etc.)
        
            RESULT (model result, Phanerozoic mean distance, distance)
        

    

In [ ]:
relative_difference_results_dict = {}
pos_relative_difference_results_dict = {}
base_distance = results_dict['gast']['ALL_ON_iter=1000_2024Apr29'][2]
keys = []
times = np.arange(-420,1,1)

#base_distance = results_dict['ALL_ON_iter=1000_2022Oct19'][3]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18,6))
#value 3 is the altered Bhattacharyya distance
for ind, (key, value) in enumerate(results_dict['gast'].items()):
    #distance between result and temperature curve is item 3
    dist = value[2]
    keys.append(key)
    #set up empty arrays
    relative_difference = np.zeros(len(dist), dtype=float)
    pos_relative_difference = np.zeros(len(dist), dtype=float)

    #loop through each timestep in the dist
    for timestep in np.arange(len(dist)):
        
        if dist[timestep] < base_distance[timestep]:
            pos_relative_difference[timestep] = ((dist[timestep]/base_distance[timestep])**-1)
            relative_difference[timestep] = (1 - (dist[timestep]/base_distance[timestep])**-1)-1
        else:
            pos_relative_difference[timestep] = (dist[timestep]/base_distance[timestep])
            relative_difference[timestep] = (dist[timestep]/base_distance[timestep])-1
     
    relative_difference_results_dict[key] = relative_difference
    pos_relative_difference_results_dict[key] = pos_relative_difference

    ax.plot(times, relative_difference, label=key, c=ordered_colors[ind])

ax.legend()
ax.grid()
#ax.set_xlim(-360,-300)
#ax.set_ylim(-10,10)

#fig.savefig('./results/figures/wasserstein_example.pdf')

In [ ]:
y = []
for i in pos_relative_difference_results_dict.values():
    y.append(savgol_filter(i, 21, 1))
    
fig, ax = plt.subplots(figsize=(9,3))

ax.stackplot(times, y, colors=ordered_colors)
ax.legend()

In [13]:
y = []
for ind, (key, value) in enumerate(relative_difference_results_dict.items()):
    y.append(relative_difference_results_dict[key])
    
    #y.append(savgol_filter(i, 21, 1))

y = np.asarray(y)

y_abs = []
for ind, (key, value) in enumerate(relative_difference_results_dict.items()):
    y_abs.append(pos_relative_difference_results_dict[key])
    
    #y.append(savgol_filter(i, 21, 1))

y_abs = np.asarray(y_abs)


In [ ]:
from matplotlib.colors import LogNorm

ylabels = []
for i in keys:
    ylabels.append('_'.join((i).split('_')[:2]))
    
fig, ax = plt.subplots(figsize=(16,4))
extent=[-420,0,5,0]
shape = y.shape
im = ax.matshow(y_abs[:], aspect=30, norm=LogNorm(vmin=1, vmax=10), cmap=cm.batlow, extent=extent)
ax.set_yticklabels(ylabels)
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Time (Ma)')

# glacials
#ax.axvspan(-445, -443, color='k', alpha=0.15)
#ax.axvspan(-330, -260, color='k', alpha=0.15)
#ax.axvspan(-35, -0, color='k', alpha=0.15)


fig.colorbar(im)
ax.grid()
fig.savefig('./results/figures/heatmap_pos_relative_change_2024Apr29.pdf')

In [ ]:
from matplotlib.colors import LogNorm

ylabels = []
for i in keys:
    ylabels.append('_'.join((i).split('_')[:2]))
    
fig, ax = plt.subplots(figsize=(16,4))
extent=[-420,0,5,0]
shape = y.shape
im = ax.matshow(y[:], aspect=30, vmin=-10, vmax=10, cmap=cm.cork, extent=extent)
ax.set_yticklabels(ylabels)
#ax.xaxis.set_ticks_position(pos)
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Time (Ma)')

# glacials
#ax.axvspan(-445, -443, color='k', alpha=0.25)
ax.axvspan(-330, -260, color='k', alpha=0.25)
ax.axvspan(-35, -0, color='k', alpha=0.25)

fig.colorbar(im)
ax.grid()
#fig.savefig('./results/figures/heatmap_relative_change_2024Apr29.pdf')

In [ ]:
#tidier plot


fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(18,12))

for ind, (key, value) in enumerate(relative_difference_results_dict.items()):
    a = savgol_filter(value, 21, 1)
    b = savgol_filter(pos_relative_difference_results_dict[key],21 ,1)
    ax1.plot(times, a, c=ordered_colors[ind], lw=4, label=key)
    ax2.plot(times, b, c=ordered_colors[ind], lw=4, label=key)
ax1.set_xlim(-540,0)
ax2.set_xlim(-540,0)
#ax2.set_ylim(1,6)
ax2.semilogy()
ax1.set_ylabel('Relative difference to default run')
ax2.set_ylabel('Relative difference to default run')
ax2.set_xlabel('Time (Ma)')
ax1.legend()
ax1.grid()
ax2.grid()

#fig.savefig('./results/figures/Phanerozoic_misfit_v4_SMOOTHED.pdf')


In [ ]:
#tidier plot


fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(18,12))

for ind, (key, value) in enumerate(relative_difference_results_dict.items()):
    a = savgol_filter(value, 21, 1)
    b = savgol_filter(pos_relative_difference_results_dict[key],21 ,1)
    ax1.plot(times, a, c=ordered_colors[ind], lw=4, label=key)
    ax2.plot(times, b, c=ordered_colors[ind], lw=4, label=key)
ax1.set_xlim(-540,0)
ax2.set_xlim(-540,0)
#ax2.set_ylim(1,6)
ax2.semilogy()
ax1.set_ylabel('Relative difference to default run')
ax2.set_ylabel('Relative difference to default run')
ax2.set_xlabel('Time (Ma)')
ax1.legend()
ax1.grid()
ax2.grid()

#fig.savefig('./results/figures/Phanerozoic_misfit_v4_SMOOTHED.pdf')
